In [1]:
import os
import pandas as pd
import numpy
import csv
import requests
from bs4 import BeautifulSoup
import re
## openpyxl-3.1.5

In [2]:
!pip install openpyxl

In [3]:
def scraper(url="https://data.nsw.gov.au/data/dataset/fuel-check"):
    download_links = []
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")

        for link in soup.find_all("a", href=True):
            href = link["href"]
            if href and (href.endswith(".csv") or href.endswith(".xlsx")) and ("24." in href or "25." in href):
                download_links.append(href)
        if download_links:
            print("Download links found (2024 or 2025):")
            for link in download_links:
                print(link)
        else:
            print("Not found")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL: {e}")
    except Exception as e:
        print(f"Error: {e}")
    return download_links

def combine(download_links):
    dataframes = []
    for url in download_links:
        if url.endswith(".xlsx"):
            df = pd.read_excel(url, engine="openpyxl")
        elif url.endswith(".csv"):
            df = pd.read_csv(url)
        else:
            continue
        dataframes.append(df)
    return dataframes
    final_df = pd.concat(dataframes, ignore_index=True)
    return final_df

def save(dataframes, loc='./final.csv'):
  #pd.concat(dataframes, ignore_index=True).to_csv(index=False)
  dataframes.to_csv(loc, sep='\t', encoding='utf-8', index=False, header=True)

In [4]:
df = combine(scraper())

Download links found (2024 or 2025):
https://data.nsw.gov.au/data/dataset/a97a46fc-2bdd-4b90-ac7f-0cb1e8d7ac3b/resource/2d785043-38eb-4eeb-b992-89f1af8d91e1/download/fuelcheck_pricehistory_jan2024.xlsx
https://data.nsw.gov.au/data/dataset/a97a46fc-2bdd-4b90-ac7f-0cb1e8d7ac3b/resource/404dcdf6-bbde-4ccf-869e-259bcc408ce5/download/fuelcheck_pricehistory_feb2024.xlsx
https://data.nsw.gov.au/data/dataset/a97a46fc-2bdd-4b90-ac7f-0cb1e8d7ac3b/resource/8c461685-3ed2-47e0-b9e6-9737c58a21bf/download/fuelcheck_pricehistory_mar2024.xlsx
https://data.nsw.gov.au/data/dataset/a97a46fc-2bdd-4b90-ac7f-0cb1e8d7ac3b/resource/56a4fdb5-0789-4a2f-a89f-18e0159e210f/download/fuelcheck_pricehistory_apr2024.xlsx
https://data.nsw.gov.au/data/dataset/a97a46fc-2bdd-4b90-ac7f-0cb1e8d7ac3b/resource/f70eb8e4-7291-47c8-874c-daea7eda0df1/download/fuelcheck_pricehistory_may2024.xlsx
https://data.nsw.gov.au/data/dataset/a97a46fc-2bdd-4b90-ac7f-0cb1e8d7ac3b/resource/7c6e7325-95ff-409e-becf-ffb45ad5271b/download/fuelcheck

In [5]:
final_df = pd.concat(df, ignore_index=True)

In [6]:
save(final_df)

In [7]:
final_df.head()

,ServiceStationName,Address,Suburb,Postcode,Brand,FuelCode,PriceUpdatedDate,Price
0,Costco Canberra Airport (Members Only),"39-41 Mustang Avenue, Canberra Airport ACT 2609",Canberra Airport,2609,Costco,U91,2024-01-01 00:44:05,168.7
1,Costco Canberra Airport (Members Only),"39-41 Mustang Avenue, Canberra Airport ACT 2609",Canberra Airport,2609,Costco,P98,2024-01-01 00:44:05,189.7
2,Costco Canberra Airport (Members Only),"39-41 Mustang Avenue, Canberra Airport ACT 2609",Canberra Airport,2609,Costco,PDL,2024-01-01 00:44:05,177.7
3,Costco Casula (Members only),"20 Parkersfarm Place, Casula NSW 2170",Casula,2170,Costco,E10,2024-01-01 00:44:11,164.7
4,Costco Casula (Members only),"20 Parkersfarm Place, Casula NSW 2170",Casula,2170,Costco,P98,2024-01-01 00:44:11,188.7


In [8]:
#final_df.to_csv('final.csv', index=False)

**Data Cleaning Section**

1. Initial Checks

In [9]:
print(final_df.head())

                       ServiceStationName  \
0  Costco Canberra Airport (Members Only)   
1  Costco Canberra Airport (Members Only)   
2  Costco Canberra Airport (Members Only)   
3            Costco Casula (Members only)   
4            Costco Casula (Members only)   

                                           Address            Suburb  \
0  39-41 Mustang Avenue, Canberra Airport ACT 2609  Canberra Airport   
1  39-41 Mustang Avenue, Canberra Airport ACT 2609  Canberra Airport   
2  39-41 Mustang Avenue, Canberra Airport ACT 2609  Canberra Airport   
3            20 Parkersfarm Place, Casula NSW 2170            Casula   
4            20 Parkersfarm Place, Casula NSW 2170            Casula   

   Postcode   Brand FuelCode     PriceUpdatedDate  Price  
0      2609  Costco      U91  2024-01-01 00:44:05  168.7  
1      2609  Costco      P98  2024-01-01 00:44:05  189.7  
2      2609  Costco      PDL  2024-01-01 00:44:05  177.7  
3      2170  Costco      E10  2024-01-01 00:44:11  164.7  
4

In [10]:
print(final_df.shape)


(1128302, 8)


In [11]:
print(final_df.describe())


           Postcode         Price
count  1.128302e+06  1.128302e+06
mean   2.337399e+03  1.962306e+02
std    2.390291e+02  1.981015e+01
min    2.007000e+03  5.590000e+01
25%    2.152000e+03  1.829000e+02
50%    2.259000e+03  1.957000e+02
75%    2.541000e+03  2.089000e+02
max    4.383000e+03  2.999000e+02


In [12]:
print(final_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128302 entries, 0 to 1128301
Data columns (total 8 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   ServiceStationName  1128302 non-null  object 
 1   Address             1128302 non-null  object 
 2   Suburb              1128302 non-null  object 
 3   Postcode            1128302 non-null  int64  
 4   Brand               1128302 non-null  object 
 5   FuelCode            1128302 non-null  object 
 6   PriceUpdatedDate    1128302 non-null  object 
 7   Price               1128302 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 68.9+ MB
None


In [13]:
print(final_df.columns)

Index(['ServiceStationName', 'Address', 'Suburb', 'Postcode', 'Brand',
       'FuelCode', 'PriceUpdatedDate', 'Price'],
      dtype='object')


2. Check Missing Values

In [14]:
print("Missing values per column:\n", final_df.isnull().sum())

Missing values per column:
 ServiceStationName    0
Address               0
Suburb                0
Postcode              0
Brand                 0
FuelCode              0
PriceUpdatedDate      0
Price                 0
dtype: int64


3. Data Cleaning Steps

3.1 Duplicates Entry

In [15]:
# Convert date column
final_df['PriceUpdatedDate'] = pd.to_datetime(final_df['PriceUpdatedDate'])

In [16]:
print(final_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128302 entries, 0 to 1128301
Data columns (total 8 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   ServiceStationName  1128302 non-null  object        
 1   Address             1128302 non-null  object        
 2   Suburb              1128302 non-null  object        
 3   Postcode            1128302 non-null  int64         
 4   Brand               1128302 non-null  object        
 5   FuelCode            1128302 non-null  object        
 6   PriceUpdatedDate    1128302 non-null  datetime64[ns]
 7   Price               1128302 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 68.9+ MB
None


In [17]:
# Duplicate Price Update per station and fuel type
duplicates = final_df.duplicated(subset=['ServiceStationName','Address', 'FuelCode', 'PriceUpdatedDate'], keep=False)
print("Duplicate entries found:", duplicates.sum())


Duplicate entries found: 38


In [18]:
# Display the actual duplicate rows
duplicate_rows = final_df[duplicates]
duplicate_rows.head(10)

,ServiceStationName,Address,Suburb,Postcode,Brand,FuelCode,PriceUpdatedDate,Price
17833,Ampol Wyong,"M1 Southbound, Wyong NSW 2259",Wyong,2259,Ampol,LPG,2024-01-08 08:34:18,105.9
17834,Ampol Wyong,"M1 Southbound, Wyong NSW 2259",Wyong,2259,Ampol,LPG,2024-01-08 08:34:18,105.9
25226,Ampol Wyong,"M1 Northbound, Wyong NSW 2259",Wyong,2259,Ampol,PDL,2024-01-10 15:46:08,188.9
25229,Ampol Wyong,"M1 Northbound, Wyong NSW 2259",Wyong,2259,Ampol,PDL,2024-01-10 15:46:08,188.9
43738,Ampol Wyong,"M1 Southbound, Wyong NSW 2259",Wyong,2259,Ampol,E10,2024-01-20 09:18:18,181.9
43739,Ampol Wyong,"M1 Southbound, Wyong NSW 2259",Wyong,2259,Ampol,P95,2024-01-20 09:18:18,198.9
43740,Ampol Wyong,"M1 Southbound, Wyong NSW 2259",Wyong,2259,Ampol,E10,2024-01-20 09:18:18,181.9
43741,Ampol Wyong,"M1 Southbound, Wyong NSW 2259",Wyong,2259,Ampol,P95,2024-01-20 09:18:18,198.9
53547,Ampol Wyong,"M1 Northbound, Wyong NSW 2259",Wyong,2259,Ampol,P95,2024-01-24 12:18:06,202.9
53551,Ampol Wyong,"M1 Northbound, Wyong NSW 2259",Wyong,2259,Ampol,P95,2024-01-24 12:18:06,202.9


In [19]:
#Average Duplicate
final_df = final_df.groupby(['ServiceStationName', 'Address','FuelCode', 'PriceUpdatedDate'], as_index=False).agg({
    'Suburb': 'first',
    'Postcode': 'first',
    'Brand': 'first',
    'Price': 'mean'
})

In [20]:
duplicates = final_df.duplicated(subset=['ServiceStationName','Address', 'FuelCode', 'PriceUpdatedDate'], keep=False)
print("Duplicate entries found:", duplicates.sum())

Duplicate entries found: 0


In [21]:
print(final_df.shape)
print(final_df.info())

(1128283, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128283 entries, 0 to 1128282
Data columns (total 8 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   ServiceStationName  1128283 non-null  object        
 1   Address             1128283 non-null  object        
 2   FuelCode            1128283 non-null  object        
 3   PriceUpdatedDate    1128283 non-null  datetime64[ns]
 4   Suburb              1128283 non-null  object        
 5   Postcode            1128283 non-null  int64         
 6   Brand               1128283 non-null  object        
 7   Price               1128283 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 68.9+ MB
None


In [22]:
final_df.head()

,ServiceStationName,Address,FuelCode,PriceUpdatedDate,Suburb,Postcode,Brand,Price
0,7 Express Carlton,"370 Railway Parade, Carlton NSW 2216",E10,2024-01-02 15:20:59,Carlton,2216,Independent,182.9
1,7 Express Carlton,"370 Railway Parade, Carlton NSW 2216",E10,2024-01-03 21:22:36,Carlton,2216,Independent,181.9
2,7 Express Carlton,"370 Railway Parade, Carlton NSW 2216",E10,2024-01-05 11:31:42,Carlton,2216,Independent,179.9
3,7 Express Carlton,"370 Railway Parade, Carlton NSW 2216",E10,2024-01-06 10:01:50,Carlton,2216,Independent,177.9
4,7 Express Carlton,"370 Railway Parade, Carlton NSW 2216",E10,2024-01-08 11:46:24,Carlton,2216,Independent,174.9


In [23]:
# Convert date column
final_df['PriceUpdatedDate'] = pd.to_datetime(final_df['PriceUpdatedDate'])

4. Anomaly Analysis

4.1 Price Anomaly

In [24]:
from scipy.stats import zscore

# Calculate z-score for the Price column
final_df['z_score'] = zscore(final_df['Price'])

# Flag anomalies (e.g., prices more than 3 standard deviations from the mean)
price_anomalies = final_df[(final_df['z_score'] > 3) | (final_df['z_score'] < -3)]
print(price_anomalies[['ServiceStationName', 'FuelCode', 'Price', 'z_score']])


         ServiceStationName FuelCode  Price   z_score
1414     7-Eleven Adamstown      LPG   99.8 -4.867763
1415     7-Eleven Adamstown      LPG   99.9 -4.862715
1416     7-Eleven Adamstown      LPG   99.9 -4.862715
1417     7-Eleven Adamstown      LPG   99.9 -4.862715
1418     7-Eleven Adamstown      LPG   99.9 -4.862715
...                     ...      ...    ...       ...
1123236    Werrington South      P98  256.9  3.062547
1123254    Werrington South      P98  256.9  3.062547
1124256  Westside Adamstown      U91  273.9  3.920696
1128193    shell long jetty      P98  285.9  4.526449
1128194    shell long jetty      P98  288.9  4.677887

[4368 rows x 4 columns]


In [25]:
plt.figure(figsize=(10, 6))
sns.boxplot(data=final_df, x='Brand', y='Price')
sns.scatterplot(data=price_anomalies, x='Brand', y='Price', color='red', label='Anomalies', zorder=10)
plt.title('Price Distribution by Brand with Anomalies')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


NameError: name 'plt' is not defined

In [ ]:
final_df['Is_Anomaly'] = final_df['z_score'].abs() > 3


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(14,6))
sns.boxplot(data=final_df, x='Brand', y='Price')
plt.title('Fuel Price Distribution by Brand (Anomalies Included)')
plt.xticks(rotation=45)
plt.show()


4.2 Sudden Price Jumps (Time Based)

In [ ]:
# First, sort by station and time
final_df = final_df.sort_values(by=['ServiceStationName', 'FuelCode', 'PriceUpdatedDate'])

# Calculate percentage change in price for each station and fuel type
final_df['Price_Pct_Change'] = final_df.groupby(['ServiceStationName', 'FuelCode'])['Price'].pct_change() * 100

# Flag rows where price increased or decreased by more than 10%
sudden_jumps = final_df[final_df['Price_Pct_Change'].abs() > 50]

# Display result
print(sudden_jumps[['ServiceStationName', 'FuelCode', 'PriceUpdatedDate', 'Price', 'Price_Pct_Change']])


In [ ]:
# Optional: Focus on a specific fuel type or brand for clarity (e.g., U91)
fuel_type = 'U91'
subset_df = final_df[final_df['FuelCode'] == fuel_type]

# Plot
plt.figure(figsize=(15, 6))
sns.lineplot(
    data=subset_df,
    x='PriceUpdatedDate',
    y='Price',
    hue='ServiceStationName',
    legend=False,
    alpha=0.3
)

# Overlay anomalies
anomalies = subset_df[subset_df['Price_Pct_Change'].abs() > 50]
sns.scatterplot(
    data=anomalies,
    x='PriceUpdatedDate',
    y='Price',
    color='red',
    s=40,
    label='>50% Change'
)

plt.title(f'Fuel Price Over Time (FuelCode: {fuel_type}) with >50% Jumps/Drops Highlighted')
plt.xlabel('Date')
plt.ylabel('Price (cents)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.legend()
plt.show()

In [ ]:
# Filter only rows with >10% price change
sudden_jumps = final_df[final_df['Price_Pct_Change'].abs() > 50].copy()

# Create a short label for clarity
sudden_jumps['Label'] = sudden_jumps['ServiceStationName'] + " | " + sudden_jumps['PriceUpdatedDate'].dt.strftime('%Y-%m-%d')

# Sort by absolute % change
sudden_jumps = sudden_jumps.sort_values(by='Price_Pct_Change', ascending=False).head(30)  # Limit to top 30 for readability

plt.figure(figsize=(14, 8))
sns.barplot(
    data=sudden_jumps,
    y='Label',
    x='Price_Pct_Change',
    hue='Brand',
    dodge=False,
    palette='coolwarm'
)
plt.axvline(x=0, color='gray', linestyle='--')
plt.title('Top 30 Sudden Fuel Price Jumps/Drops (>50%)')
plt.xlabel('Price % Change')
plt.ylabel('Station | Date')
plt.tight_layout()
plt.legend(title='Brand')
plt.show()


In [ ]:
import pandas as pd

# Step 1: Calculate the frequency of updates for each station
update_counts = final_df.groupby('ServiceStationName')['PriceUpdatedDate'].nunique()

# Step 2: Calculate mean and std of the update frequencies
mean_updates = update_counts.mean()
std_updates = update_counts.std()

# Step 3: Flag stations with unusually high or low update frequencies
high_frequency_stations = update_counts[update_counts > mean_updates + 2 * std_updates]
low_frequency_stations = update_counts[update_counts < mean_updates - 2 * std_updates]

# Combine results
unusual_frequency_stations = pd.concat([high_frequency_stations, low_frequency_stations])

# Display the stations with unusual update frequencies
print("Stations with unusually high or low update frequencies:")
print(unusual_frequency_stations)

# Optional: Display stations with high update frequencies
print("\nStations with unusually high update frequencies:")
print(high_frequency_stations)

# Optional: Display stations with low update frequencies
print("\nStations with unusually low update frequencies:")
print(low_frequency_stations)
